In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
songs = pd.read_csv('songdata.csv')
songs

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \nLet the angels fly l...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \nMore power \nPower to...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \nis something i'll believe \nf...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \nam i frightened \nwhere can ...


In [2]:
df=songs

In [3]:
df=df[df['song']=='Only Over You']
t=df['text'].tolist()

In [4]:
print(t)

["I'm out of my mind  \nAnd it's only over you  \nI'm out of my mind  \nAnd it's only over you  \nPeople think I'm crazy  \n  \nBut they don't know  \nThought love had failed me  \nBut now, they're watching it grow  \nAngel, please don't go  \nI miss you when you're gone  \nThey say I'm a silly girl  \n  \nBut I'm not a fool  \nPeople say they know me  \nBut they don't see  \nMy heart's your future  \nYour future is me  \nAngel, please don't go  \nI miss you when you're gone  \nThey say I'm a silly girl  \n  \nBut I'm not a fool  \nI'm out of my mind  \nAnd it's only over you  \nI'm out of my mind  \nAnd it's only over you\n\n"]


In [5]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)
songs['text'] = songs['text'].str.replace(r'\n', '')
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')
tfidf_matrix = tfidf.fit_transform(songs['text'])

In [6]:
tfidf_matrix

<5000x23399 sparse matrix of type '<class 'numpy.float64'>'
	with 267175 stored elements in Compressed Sparse Row format>

In [7]:
import numpy as np
from scipy.sparse import csr_matrix

# Get the array of values stored in the matrix
values = tfidf_matrix.data

# Get the array of column indices for the values
indices = tfidf_matrix.indices

# Iterate over the values and indices, and print the values
for value, index in zip(values, indices):
    print(value, index)

0.03209875270159161 4064
0.06308768646832488 6174
0.10782145087161367 7852
0.08385384042246423 18109
0.14329135478363408 2520
0.17216865826104275 2253
0.04848976547391137 19085
0.06530959140478486 6334
0.07615672778107643 9289
0.05029034101623921 9708
0.11086391389364537 9850
0.07589197082157247 3364
0.08643407087377952 15754
0.050656118525814184 14537
0.04373249282427963 9269
0.07790288484823511 23004
0.09113581744840801 9237
0.11478633093138237 14068
0.055560477022293345 5141
0.07913251802969286 22454
0.12031466724617965 14629
0.11478633093138237 15790
0.0903564879115165 6990
0.07880395863745858 795
0.09479347752664101 17009
0.08090573455898223 20821
0.11086391389364537 3915
0.04365832461784866 9589
0.03003474774719609 8615
0.11451044215201578 7255
0.12031466724617965 3322
0.1478817089984271 17929
0.08837235952651266 7064
0.15025443180782833 19769
0.32306112740764753 9396
0.1492804314847656 3202
0.09017330460101589 10338
0.10800108559447134 5248
0.12381974139072795 2819
0.09837069751

0.09356506467065083 1676
0.052051701602929226 18060
0.04966571108102239 7174
0.13940671561956333 12615
0.03374258757892933 21072
0.04720167736110125 3182
0.041441192128954855 10871
0.03670470336697243 23058
0.05498927623300188 9337
0.06915551609186774 21438
0.05230477761824107 22437
0.035279381298489315 17612
0.044102046826219876 3883
0.05402695624038636 537
0.046920023542206216 12028
0.03160653978579071 11202
0.04107887826350581 4435
0.03408622987525994 23194
0.08379360086290812 5080
0.39303364299033106 6617
0.03670470336697243 7439
0.06105019177689324 8538
0.032952354022401216 12574
0.12795531138986244 12927
0.036408725490394485 17829
0.13837738024128457 22569
0.038356882764193204 12284
0.029449826181878202 20032
0.25902877723038786 21416
0.04288621778772799 794
0.023094516319431192 20582
0.06273985216868651 23059
0.029011382031147042 15220
0.03805051760358293 13144
0.02244105597058948 13797
0.03197393668169302 23267
0.07315325022216713 16991
0.09819394984875847 20917
0.0569374335068

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities = cosine_similarity(tfidf_matrix) 
similarities = {}
for i in range(len(cosine_similarities)):
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], 
                                           songs['artist'][x]) for x in similar_indices][1:]

In [9]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix
    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)       
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------") 
    def recommend(self, recommendation):
            song = recommendation['song']
            number_songs = recommendation['number_songs']
            recom_song = self.matrix_similar[song][:number_songs]
            self._print_message(song=song, recom_song=recom_song)       

In [10]:
recommedations = ContentBasedRecommender(similarities)


recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 5
}

recommedations.recommend(recommendation)

The 5 recommended songs for Every Day People are:
Number 1:
Alone by Bee Gees with 0.209 similarity score
--------------------
Number 2:
Daddy's Gonna Pay For Your Crashed Car by U2 with 0.185 similarity score
--------------------
Number 3:
People Are People by Depeche Mode with 0.181 similarity score
--------------------
Number 4:
Love Rescue Me by U2 with 0.181 similarity score
--------------------
Number 5:
A Different Beat by Little Mix with 0.149 similarity score
--------------------


In [15]:
recommendation2 = {
    "song": 'All The World',
    "number_songs": 4 
}
recommedations.recommend(recommendation2)

The 4 recommended songs for All The World are:
Number 1:
All The Love In The World by Nine Inch Nails with 0.409 similarity score
--------------------
Number 2:
We Have All The Time In The World by Louis Armstrong with 0.377 similarity score
--------------------
Number 3:
Out Of This World by Kiss with 0.312 similarity score
--------------------
Number 4:
Desert Lullaby by Queen Adreena with 0.276 similarity score
--------------------
